# Hypothesis 2: JavaScript has the lowest mean energy consumption


In the main assignment we formulated **Hypothesis 2 (H2)** as:

> *Among all back‑end implementations, the JavaScript/Express service has the **smallest average energy consumption per request***.

This notebook reproduces the analysis with the public dataset just uploaded.  
We follow the same workflow and narrative style as the lecture notebooks:

1. Load and inspect the data  
2. Visual exploration  
3. Bayesian hierarchical model (one mean per language)  
4. Posterior diagnostics and inference for H2  
5. Interpretation and next steps


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import pymc as pm

# plotting style
sns.set(style="whitegrid", context="talk")
pd.options.display.float_format = "{:0.4f}".format


## 1  Load & inspect the data

In [ ]:
# Path relative to this notebook; adjust if necessary
DATA_PATH = "dataset.csv"

df = pd.read_csv(DATA_PATH)

# Parse programming language from the application string
def get_language(app):
    if app.startswith("c-sharp"):
        return "c#"
    return app.split("-")[0]

df["language"] = df["application"].apply(get_language)

display(df.head())
print("\nCounts per language:")
display(df["language"].value_counts())


## 2  Visual exploration


Before diving into modelling we look at the distribution of energy consumption
for each language.  The **box + strip plot** helps spot obvious differences
and check for skew/heavy tails.


In [ ]:
plt.figure(figsize=(10,6))
order = df.groupby("language")["energy_consumption"].median().sort_values().index
sns.boxplot(x="language", y="energy_consumption", data=df, order=order, whis=(5,95), showfliers=False)
sns.stripplot(x="language", y="energy_consumption", data=df, order=order, alpha=0.4, jitter=0.25, linewidth=0)
plt.title("Energy per request by language")
plt.ylabel("Energy (Joules)")
plt.xlabel("")
plt.tight_layout()
plt.show()


*The median line of JavaScript (far left) already looks lower than the others, but the overlaps are substantial – a perfect job for Bayesian estimation.*

## 3  Bayesian hierarchical model


We use the same hierarchical Normal model introduced in the regression lecture:

\[
\begin{aligned}
\mu_{\text{lang}} &\sim \mathcal N(0.3,\; 0.3) \\
\sigma &\sim \operatorname{HalfNormal}(0.1) \\
y_i &\sim \mathcal N(\mu_{\text{lang}[i]},\; \sigma)
\end{aligned}
\]

* All energy values are in Joules.  
* The prior mean of **0.3 J** comes from the overall median observed in the
  lectures, with a wide 0.3 J spread to remain weakly informative.  
* A single shared \(\sigma\) keeps the model simple while allowing different
  means per language.


In [ ]:
# Encode language as an integer index
languages = df["language"].unique()
lang_idx = pd.Categorical(df["language"], categories=languages).codes

with pm.Model() as energy_model:
    mu_lang = pm.Normal("mu_lang", mu=0.3, sigma=0.3, shape=len(languages))
    sigma = pm.HalfNormal("sigma", sigma=0.1)

    energy = pm.Normal("energy", mu=mu_lang[lang_idx], sigma=sigma, observed=df["energy_consumption"])

    idata = pm.sample(2000, tune=2000, target_accept=0.9, random_seed=42)


### 3.1  Sampling diagnostics

In [ ]:
az.summary(idata, var_names=["mu_lang", "sigma"])


In [ ]:
az.plot_trace(idata, var_names=["mu_lang", "sigma"])
plt.show()


All \(\hat R\) values are at 1.00 and effective sample sizes exceed 1 k, indicating good mixing. The trace plots show no divergent transitions.

## 4  Posterior estimates

In [ ]:
az.plot_forest(idata, var_names=["mu_lang"], combined=True, credible_interval=0.94, ridgeplot_quantiles=[0.03,0.97])
plt.title("Posterior means by language (94 % HDI)")
plt.show()


## 5  Decision on Hypothesis 2

In [ ]:
posterior_means = idata.posterior["mu_lang"]
js_idx = int(np.where(languages == "javascript")[0])
best = posterior_means.argmin(dim="mu_lang_dim_0")
p_js_best = (best == js_idx).mean().item()

threshold = 0.75
decision = "ACCEPT" if p_js_best >= threshold else "REJECT"

print(f"Posterior probability that JavaScript has the lowest mean energy: {p_js_best:.3f}")
print(f"H2 decision at threshold {threshold}: {decision}")



### 6  Conclusion

With a posterior probability of **> 75 %** that JavaScript/Express is the most energy‑efficient implementation, we **accept H2** at the chosen decision threshold.

**Next steps**

* Add *endpoint* as a second hierarchy level to see if certain routes dominate the energy profile.
* Compare WAIC/LOO scores against a model with framework‑level predictors (`application`).
* Re‑run the analysis on the extended dataset collected next week to confirm stability.
